# Real-time Face Mask Detector

## Importing the libraries

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

## Loading the trained model

In [2]:
model = load_model("model.h5")

In [3]:
def classify(image):
    
    # Image Preprocessing
    image = cv2.resize(image, (64, 64))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    # Predicting the class
    proba = model.predict(image)
    label = proba > 0.5
    label = int(label[0][0])
    
    p = proba[0][0]
    if(proba <= 0.5):
        p = 1 - p
    
    p = p * 100
    p = np.round(p, 2)
    
    return p, label

## Loading the Haar Cascade font face detector

In [4]:
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Web Cam

In [5]:
color = {1: (71, 99, 255), 0: (0, 255, 0)}
text = {1: "No Mask", 0: "Mask"}

# Use camera 0
cap = cv2.VideoCapture(0) 

while True:
    # Get the next frame
    ret, im = cap.read()
    
    # Flip the frame
    im = cv2.flip(im, 1)
    
    # Converting the frame to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (5,5),0)
    
    # Detect Multi faces 
    faces = classifier.detectMultiScale(gray)
    
    
    label = ""
    face_img = np.array([])
    
    for f in faces:
        x, y, w, h = f
        
        # Extract face
        face = im[y:y+h, x:x+w]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        
        if face_img.ndim > 0:
            p, label = classify(face)
            
        frame_text = text[label] #+ ' ' + str(p) 

        # Draw a labeled rectangle around each face in the frame
        cv2.rectangle(im, (x,y), (x+w,y+h), color[label], 2)
        cv2.rectangle(im, (x-1,y-40), (x+w+1,y), color[label], -2)
        cv2.putText(im, frame_text, (x+2, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8 ,(255,255,255), 2)
    
    # Display the frame
    cv2.imshow('Real-time Face Mask Detector', im)
    key = cv2.waitKey(1)
    
    # Break out of the loop if Esc key is pressd
    if key == 27:
        break
        
# Stop video
cap.release()

# Close all windows
cv2.destroyAllWindows()